In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 pandas
!pip install unidic
!python -m unidic download

In [ ]:
import pandas as pd
import MeCab

# MeCabを使って動詞を辞書形に修正する

# 入力CSVファイルと出力CSVファイルのパス
input_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b.csv" # 元のCSVファイル
output_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b-fixed.csv"  # 修正後のCSVファイル

# MeCabの初期化（辞書パス指定なしでunidicを使用）
tagger = MeCab.Tagger()

# CSVファイルを読み込む
try:
    df = pd.read_csv(input_csv)
    print(f"読み込んだCSVの行数: {len(df)}")

    # Extracted_Phraseの動詞を辞書形に修正
    def convert_to_base_form(phrase):
        tokens = tagger.parse(phrase).splitlines()
        base_phrase = []  # 修正後のフレーズ
        for token in tokens:
            if '\t' not in token:  # EOSや空行をスキップ
                continue
            surface, features = token.split('\t')
            features = features.split(',')
            pos = features[0]  # 品詞
            base_form = features[6] if len(features) > 6 and features[6] != '*' else surface
            # 名詞や助詞はそのまま、動詞のみ基本形に置き換える
            if pos == '動詞':
                base_phrase.append(base_form)
            else:
                base_phrase.append(surface)
        return ''.join(base_phrase)

    # DataFrameのExtracted_Phrase列を変換
    df['Extracted_Phrase'] = df['Extracted_Phrase'].apply(convert_to_base_form)

    # 修正後のデータを新しいCSVファイルに保存
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"修正後のCSVファイルを保存しました: {output_csv}")

except Exception as e:
    print(f"エラーが発生しました: {e}")
